#install

In [1]:
# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
python3-gdal is already the newest version (2.2.3+dfsg-2).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-rtree is already the newest version (0.8.3+ds-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
  Cloning git://github.com/geopandas/geopandas.git to /tmp/pip-req-build-vgyd2jl6
  Running command git clone -q git://github.com/geopandas/geopandas.git /tmp/pip-req-build-vgyd2jl6


In [2]:
from shapely.geometry import Point,Polygon
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt 
from google.colab import drive
import glob
import re
import os
from os.path import exists
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#read

In [3]:
district_NE_path = "/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/District_NE.geojson"
Province_Boundary = "/content/drive/My Drive/2 Boundary/1-1_Province_Boundary_-_mitrearth.shp"

In [4]:
district_NE = gpd.read_file(district_NE_path)
province = gpd.read_file(Province_Boundary)

In [5]:
province_NE = province[province.PART == 'NE']

#Algorithm for province

In [6]:
def check_district(lon,lat):
  point = Point(lon,lat)
  for index in province_NE.index:
    if point.within(province_NE.loc[index,'geometry']):
      return True
  return False

In [7]:
check_district(103.423698,15.916181)

True

#Filter NE

In [8]:
!ls "/content/drive/My Drive/gps_copy/2019-01"

2019-01-01_00.zip  2019-01-08_18.zip  2019-01-16_12.zip  2019-01-24_06.zip
2019-01-01_03.zip  2019-01-08_21.zip  2019-01-16_15.zip  2019-01-24_09.zip
2019-01-01_06.zip  2019-01-09_00.zip  2019-01-16_18.zip  2019-01-24_12.zip
2019-01-01_09.zip  2019-01-09_03.zip  2019-01-16_21.zip  2019-01-24_15.zip
2019-01-01_12.zip  2019-01-09_06.zip  2019-01-17_00.zip  2019-01-24_18.zip
2019-01-01_15.zip  2019-01-09_09.zip  2019-01-17_03.zip  2019-01-24_21.zip
2019-01-01_18.zip  2019-01-09_12.zip  2019-01-17_06.zip  2019-01-25_00.zip
2019-01-01_21.zip  2019-01-09_15.zip  2019-01-17_09.zip  2019-01-25_03.zip
2019-01-02_00.zip  2019-01-09_18.zip  2019-01-17_12.zip  2019-01-25_06.zip
2019-01-02_03.zip  2019-01-09_21.zip  2019-01-17_15.zip  2019-01-25_09.zip
2019-01-02_06.zip  2019-01-10_00.zip  2019-01-17_18.zip  2019-01-25_12.zip
2019-01-02_09.zip  2019-01-10_03.zip  2019-01-17_21.zip  2019-01-25_15.zip
2019-01-02_12.zip  2019-01-10_06.zip  2019-01-18_00.zip  2019-01-25_18.zip
2019-01-02_15.zip  2019-0

In [9]:
path =  '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/gps_log_NE_2019_01'

In [10]:
days = range(1,32)
month = 1
for day in days:
  export_filename = path + f'/2019_{month}_{day}.csv'

  #### check file exists
  if os.path.exists(export_filename):
    print(f'gps_log_2019_{month}_{day}.csv already taken!!')
    continue
  else:
    with open(export_filename, 'w') as fp:
      pass
    if os.path.exists(export_filename):
      print(f'gps_log_2019_{month}_{day}.csv creating...')
      pass

  #### read gps log zip
  filenames = glob.glob('/content/drive/My Drive/gps_copy/2019-01/*')
  filenames = [f for f in filenames if re.search(re.compile('/2019-0?('+ str(month) +')-0?(' + str(day) + ')_[0-2][0-9]\.zip$'), f)]
  filenames = sorted(filenames)

  dfs = []
  for filename in filenames:
    gps = pd.read_csv(filename, compression='zip')

    # clean
    gps = gps[gps['unit_type'] != 1.0]
    gps = gps.dropna()
    gps = gps.drop_duplicates()
    gps = gps.drop_duplicates(subset=['time_stamp', 'unit_id'], keep="last")
    gps = gps.loc[gps['lat'] > 14.112976]
    gps = gps.loc[gps['lon'] > 100.832301]

    # filter
    msk_NE = []
    for index,row in gps.iterrows():
     if check_district(row.lon,row.lat):
       msk_NE.append(True)
     else:
       msk_NE.append(False)

    gps = gps[msk_NE]

    #add time to 1 day 
    dfs.append(gps)

  #export  
  gps = pd.concat(dfs, ignore_index=True)
  print(f'count 2019-{month}-{day} : ',len(gps))
  
  gps.to_csv(export_filename, encoding='utf-8', index = False)

gps_log_2019_1_1.csv creating...


KeyboardInterrupt: ignored